In [1]:
!pip install -U odp-sdk --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
azure-cli-core 2.68.0 requires microsoft-security-utilities-secret-masker~=1.0.0b2, which is not installed.
azure-cli-core 2.68.0 requires argcomplete~=3.5.2, but you have argcomplete 3.6.0 which is incompatible.
azure-cli-core 2.68.0 requires knack~=0.11.0, but you have knack 0.12.0 which is incompatible.
azure-cli-core 2.68.0 requires msal[broker]==1.31.1, but you have msal 1.32.0 which is incompatible.
azure-cli-core 2.68.0 requires msal-extensions==1.2.0, but you have msal-extensions 1.3.1 which is incompatible.
awscli 2.24.27 requires cryptography<43.0.2,>=40.0.0, but you have cryptography 44.0.2 which is incompatible.


In [2]:
# Standard library imports
import json
import pandas as pd
import geojson

import shapely
from shapely import wkt
from shapely.geometry import box, MultiPolygon, Polygon
import geopandas as gpd

from odp.client import OdpClient  # The SDK

In [3]:
client = OdpClient()

In [4]:
## Request the dataset from the catalog using the UUID:
proSeas_dataset = client.catalog.get(("3e32fd06-4eb7-4da2-9acb-dd0ecb58aa88"))
proSeas_dataset.metadata.display_name

'ProtectedSeas Navigator - Comprehensive Database of Marine Life Protections'

In [5]:
proSeas_data = client.table_v2(proSeas_dataset)

## Create a geographic query and search for specific types of MPAs

In [6]:
## Bounding box for Southern Norway
## Enter min and max latitude and longitude values to create a bounding box polygon below. Or use the structure below to add any Well-Known-Text or GeoJSON defined polygon.
lat_min = 56
lat_max = 62
lon_min = 8
lon_max = 12
query_geometry = box(lon_min, lat_min, lon_max, lat_max).wkt
query_geometry

'POLYGON ((12 56, 12 62, 8 62, 8 56, 12 56))'

### Fetch data from ODP 

In [7]:
# Find Natura2000 sites within the search area
df_geo = pd.concat(proSeas_data.select(f"geometry within '{query_geometry}' AND boundary_source == 'Natura2000'").dataframes(), ignore_index=True)
df_geo.head()

,geometry_reduced,dredging_prohibited,gillnets_entangling_nets,boundary_source,last_update,dip_scoop_nets,landing_prohibited,season,mooring,spear_fishing,...,url,site_major_version,misc_gear,longlining,traps_n_pots,removal_of_historic_artifacts_prohibited,construction_prohibited,diving_prohibited,recreational_restrictions,gillnetting
0,"MULTIPOLYGON Z (((8.507997 56.744095 0, 8.5084...",3.0,NaN,Natura2000,2021-08-03,NaN,3.0,Year-round,NaN,3.0,...,MPA Website|https://natura2000.eea.europa.eu/N...,1,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
1,"MULTIPOLYGON Z (((8.444085 57.014151 0, 8.4448...",3.0,NaN,Natura2000,2021-08-03,NaN,3.0,Year-Round,NaN,3.0,...,https://natura2000.eea.europa.eu/Natura2000/SD...,1,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2,"MULTIPOLYGON Z (((9.463926 57.191341 0, 9.4909...",3.0,NaN,Natura2000,2021-08-03,NaN,3.0,Year-Round,NaN,3.0,...,https://natura2000.eea.europa.eu/Natura2000/SD...,1,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
3,"MULTIPOLYGON Z (((9.247454 56.569421 0, 9.2468...",3.0,NaN,Natura2000,2021-08-03,NaN,3.0,Year-round,NaN,3.0,...,MPA Website|https://natura2000.eea.europa.eu/N...,1,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
4,"MULTIPOLYGON Z (((9.19706 56.929287 0, 9.19709...",3.0,NaN,Natura2000,2023-01-26,NaN,3.0,Year-round,NaN,3.0,...,MPA Website|https://natura2000.eea.europa.eu/N...,1,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


## Export the data in your prefered file format

### Pandas

In [8]:
# Uncomment the line below to select the output format

# Export to CSV
df_geo.to_csv('df_country.csv', index=False)

# Export to JSON
# df_country.to_json('df_country.json', orient='records', lines=True)

# Export to Parquet
# df_country.to_parquet('df_country.parquet', index=False)

### GeoPandas

In [9]:
# Function to convert WKT string to 2D Shapely geometry
def convert_wkt_to_2d(geometry_wkt):
    if not geometry_wkt:  # Check if it's None or empty
        return None  

    try:
        geom = wkt.loads(geometry_wkt)  # Convert text to Shapely geometry
        if geom and hasattr(geom, "has_z") and geom.has_z:  # Ensure geom exists and has Z
            # Remove Z by keeping only X, Y coordinates
            return MultiPolygon([
                Polygon([(x, y) for x, y, *_ in polygon.exterior.coords])
                for polygon in geom.geoms
            ]) if isinstance(geom, MultiPolygon) else Polygon([(x, y) for x, y, *_ in geom.exterior.coords])
        
        return geom  # Already 2D
    except Exception as e:
        print(f"Error converting geometry: {e}")
        return None  # Return None instead of breaking

In [10]:
# Convert DataFrame to GeoDataFrame
df_geo['geometry'] = df_geo['geometry'].apply(convert_wkt_to_2d)
gdf = gpd.GeoDataFrame(df_geo, geometry='geometry', crs="EPSG:4326")

In [11]:
gdf.to_file("df_geo.geojson", driver="GeoJSON")
print("GeoJSON saved successfully!")

GeoJSON saved successfully!
